# TODO: Title

This notebook lists all the steps that you need to complete the complete this project. You will need to complete all the TODOs in this notebook as well as in the README and the two python scripts included with the starter code.


**TODO**: Give a helpful introduction to what this notebook is for. Remember that comments, explanations and good documentation make your project informative and professional.

**Note:** This notebook has a bunch of code and markdown cells with TODOs that you have to complete. These are meant to be helpful guidelines for you to finish your project while meeting the requirements in the project rubrics. Feel free to change the order of these the TODO's and use more than one TODO code cell to do all your tasks.

In [53]:
# TODO: Install any packages that you might need
# For instance, you will need the smdebug package
!pip install smdebug 

In [2]:
# TODO: Import any packages that you might need
# For instance you will need Boto3 and Sagemaker
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile
import os

## Dataset
TODO: Explain what dataset you are using for this project. Maybe even give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understand of it.

In [11]:
#TODO: Fetch and upload the data to AWS S3

# Command to download and unzip data
# !wget https://s3-us-west-1.amazonaws.com/udacity-aind/dog-project/dogImages.zip
# !unzip dogImages.zip

## Hyperparameter Tuning
**TODO:** This is the part where you will finetune a pretrained model with hyperparameter tuning. Remember that you have to tune a minimum of two hyperparameters. However you are encouraged to tune more. You are also encouraged to explain why you chose to tune those particular hyperparameters and the ranges.

**Note:** You will need to use the `hpo.py` script to perform hyperparameter tuning.

In [17]:
#TODO: Declare your HP ranges, metrics etc.
hyperparameter_ranges = {
    "learning_rate": ContinuousParameter(0.001, 0.1),
    "batch_size": CategoricalParameter([32, 64, 128, 256, 512]),
}

role = sagemaker.get_execution_role()
bucket = "sagemaker-us-east-2-285132913936"

objective_metric_name = "Test Loss"
objective_type = "Minimize"
metric_definitions = [{"Name": "Test Loss", "Regex": "Testing Loss: ([0-9\\.]+)"}]

In [18]:
#TODO: Create estimators for your HPs

# estimator = # TODO: Your estimator here

# tuner = # TODO: Your HP tuner here
estimator = PyTorch(
    entry_point="hpo.py",
    base_job_name='dog_classifier',
    role=role,
    framework_version="1.8.0",
    instance_count=1,
    instance_type="ml.m5.xlarge",
    py_version='py3'
)

tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=2,
    max_parallel_jobs=2,
    objective_type=objective_type
)

In [21]:
# TODO: Fit your HP Tuner
os.environ['SM_CHANNEL_TRAINING']=f's3://{bucket}/dogImages/'
os.environ['SM_MODEL_DIR']=f's3://{bucket}/dogImages/model/'
os.environ['SM_OUTPUT_DATA_DIR']=f's3://{bucket}/dogImages/output/'
tuner.fit({"training": f"s3://{bucket}/dogImages/"})


.....................................................................................................................................................................................................................................................................................................!


In [129]:
# TODO: Get the best estimators and the best HPs

best_estimator = tuner.best_estimator()

#Get the hyperparameters of the best trained model

hyperparameters = {"batch_size": int(best_estimator.hyperparameters()['batch_size'].replace('"', '')), \
                   "learning_rate": best_estimator.hyperparameters()['learning_rate']}


2022-01-11 01:43:07 Starting - Preparing the instances for training
2022-01-11 01:43:07 Downloading - Downloading input data
2022-01-11 01:43:07 Training - Training image download completed. Training in progress.
2022-01-11 01:43:07 Uploading - Uploading generated training model
2022-01-11 01:43:07 Completed - Training job completed


## Model Profiling and Debugging
TODO: Using the best hyperparameters, create and finetune a new model

**Note:** You will need to use the `train_model.py` script to perform model profiling and debugging.

In [130]:
# TODO: Set up debugging and profiling rules and hooks
rules = [
    Rule.sagemaker(rule_configs.vanishing_gradient()),
#     Rule.sagemaker(rule_configs.loss_not_decreasing()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]

hook_config = DebuggerHookConfig(
    hook_parameters={"train.save_interval": "1",
                     "eval.save_interval": "1"}
)

profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, framework_profile_params=FrameworkProfile(num_steps=1)
)




In [121]:
# TODO: Create and fit an estimator

estimator = PyTorch(
    entry_point='train_model.py',
    base_job_name='debug-profile',
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    framework_version='1.8.0',
    py_version='py3',
    hyperparameters=hyperparameters,
    ## Debugger and Profiler parameters
    rules = rules,
    debugger_hook_config=hook_config,
    profiler_config=profiler_config,
)

In [122]:
# TODO: Plot a debugging output.
estimator.fit({"training": f"s3://{bucket}/dogImages/"}, wait=False)
session = boto3.session.Session()
region = session.region_name

training_job_name = estimator.latest_training_job.name
print(f"Training jobname: {training_job_name}")
print(f"Region: {region}")

from smdebug.trials import create_trial
from smdebug.core.modes import ModeKeys

trial = create_trial(estimator.latest_job_debugger_artifacts_path())


print(trial.tensor_names())
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.TRAIN)))
print(len(trial.tensor("CrossEntropyLoss_output_0").steps(mode=ModeKeys.EVAL)))

Training jobname: debug-profile-2022-01-11-08-11-11-962
Region: us-east-2
[2022-01-11 08:11:12.463 ip-172-16-35-0:20812 INFO s3_trial.py:42] Loading trial debug-output at path s3://sagemaker-us-east-2-285132913936/debug-profile-2022-01-11-08-11-11-962/debug-output
[2022-01-11 08:11:32.777 ip-172-16-35-0:20812 WARNING trial.py:148] Waiting to read collections files generated by the training job,from s3://sagemaker-us-east-2-285132913936/debug-profile-2022-01-11-08-11-11-962/debug-output/. If this has been a while, you might want to check that the trial is pointed at the right path.
[2022-01-11 08:11:34.808 ip-172-16-35-0:20812 WARNING trial.py:148] Waiting to read collections files generated by the training job,from s3://sagemaker-us-east-2-285132913936/debug-profile-2022-01-11-08-11-11-962/debug-output/. If this has been a while, you might want to check that the trial is pointed at the right path.
[2022-01-11 08:11:36.839 ip-172-16-35-0:20812 WARNING trial.py:148] Waiting to read colle

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [127]:
# TODO: Display the profiler output
from smdebug.profiler.analysis.notebook_utils.training_job import TrainingJob

tj = TrainingJob(training_job_name, region)
tj.wait_for_sys_profiling_data_to_be_available()
from smdebug.profiler.analysis.notebook_utils.timeline_charts import TimelineCharts

system_metrics_reader = tj.get_systems_metrics_reader()
system_metrics_reader.refresh_event_file_list()

view_timeline_charts = TimelineCharts(
    system_metrics_reader,
    framework_metrics_reader=None,
    select_dimensions=["CPU", "GPU"],
    select_events=["total"],
)

rule_output_path = estimator.output_path + estimator.latest_training_job.job_name + "/rule-output"
print(f"You will find the profiler report in {rule_output_path}")

ProfilerConfig:{'S3OutputPath': 's3://sagemaker-us-east-2-285132913936/', 'ProfilingIntervalInMilliseconds': 500, 'ProfilingParameters': {'DataloaderProfilingConfig': '{"StartStep": 0, "NumSteps": 1, "MetricsRegex": ".*", }', 'DetailedProfilingConfig': '{"StartStep": 0, "NumSteps": 1, }', 'FileOpenFailThreshold': '50', 'HorovodProfilingConfig': '{"StartStep": 0, "NumSteps": 1, }', 'LocalPath': '/opt/ml/output/profiler', 'PythonProfilingConfig': '{"StartStep": 0, "NumSteps": 1, "ProfilerName": "cprofile", "cProfileTimer": "total_time", }', 'RotateFileCloseIntervalInSeconds': '60', 'RotateMaxFileSizeInBytes': '10485760', 'SMDataParallelProfilingConfig': '{"StartStep": 0, "NumSteps": 1, }'}}
s3 path:s3://sagemaker-us-east-2-285132913936/debug-profile-2022-01-11-08-11-11-962/profiler-output


Profiler data from system is available
select events:['total']
select dimensions:['CPU', 'GPU']
filtered_events:{'total'}
filtered_dimensions:{'CPUUtilization-nodeid:algo-1'}


You will find the profiler report in s3://sagemaker-us-east-2-285132913936/debug-profile-2022-01-11-08-11-11-962/rule-output


## Model Deploying

In [125]:
# TODO: Deploy your model to an endpoint
model_location = "s3://sagemaker-us-east-2-285132913936/pytorch-training-220111-0120-001-7d507401/output/model.tar.gz"
predictor=estimator.deploy(initial_instance_count=1, instance_type='ml.m5.large') # TODO: Add your deployment configuration like instance type and number of instances

------!

In [134]:
# Try predictor 
import sagemaker
import boto3
from sagemaker.tuner import CategoricalParameter, ContinuousParameter, HyperparameterTuner
from sagemaker.pytorch import PyTorch
from sagemaker import get_execution_role
from sagemaker.debugger import Rule, DebuggerHookConfig, TensorBoardOutputConfig, CollectionConfig, ProfilerRule, rule_configs
from sagemaker.debugger import ProfilerConfig, FrameworkProfile

from sagemaker.pytorch import PyTorchModel
from sagemaker.predictor import Predictor
from sagemaker.pytorch.model import PyTorchPredictor

jpeg_serializer = sagemaker.serializers.IdentitySerializer("image/jpeg")
json_deserializer = sagemaker.deserializers.JSONDeserializer()


class ImagePredictor(Predictor):
    def __init__(self, endpoint_name, sagemaker_session):
        super(ImagePredictor, self).__init__(
            endpoint_name,
            sagemaker_session=sagemaker_session,
            serializer=jpeg_serializer,
            deserializer=json_deserializer,
        )

pytorch_predictor = PyTorchPredictor(endpoint_name = "debug-profile-2022-01-11-14-05-09-139", 
                                     sagemaker_session = sagemaker.session.Session(), 
                                    serializer=jpeg_serializer,
                                    deserializer=json_deserializer
                                    )

# pytorch_model = PyTorchModel(model_data=model_location, role=role, py_version='py3',
#                              framework_version='1.4',
#                              predictor_cls=ImagePredictor)

In [ ]:
# TODO: Run an prediction on the endpoint
import requests
import torchvision.transforms as transforms
from io import BytesIO
from PIL import Image

# request_dict={ "url": "https://cdn1-www.cattime.com/assets/uploads/2011/12/file_2744_british-shorthair-460x290-460x290.jpg" }
print(1)
request_dict={ "url": "https://www.akc.org/wp-content/uploads/2016/06/German-Shepherd-Dog-laying-down-in-the-backyard.jpeg" }
image = requests.get(request_dict['url']).content# TODO: Your code to load and preprocess image to send to endpoint for prediction
print(2)
# test_transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     ])
# print(3)
# image = Image.open(BytesIO(image))
print(4)
response = pytorch_predictor.predict(image)


1
2
4


In [ ]:
# TODO: Remember to shutdown/delete your endpoint once your work is done
predictor.delete_endpoint()